<a href="https://colab.research.google.com/github/Alimv81/Dog-or-cat/blob/main/simple_pretrained_dog_or_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [ ]:
# Constants
input_shape = (224, 224, 3)
num_classes = 2  # Cats and Dogs

# Load pre-trained MobileNetV2 model (without the top fully connected layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
# Freeze base model layers to avoid overfitting on a small dataset
base_model.trainable = False

# Add custom classification head for binary classification (cats vs. dogs)
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Download and prepare the dataset
dataset_name = 'cats_vs_dogs'
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name,
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

In [ ]:
# Data augmentation and preprocessing
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

ds_train = ds_train.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Fine-tune the model on the dataset
model.fit(ds_train, epochs=6)

582/582 [==============================] - 770s 1s/step - loss: 0.0674 - accuracy: 0.9791


In [ ]:
# evaluating the accuracy of the model
evaluation = model.evaluate(ds_test)

146/146 [==============================] - 200s 1s/step - loss: 0.0347 - accuracy: 0.9884


In [ ]:
# Save the model for future use
model.save('cat_dog_classifier.h5')